In [1]:
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
# word tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#stemmers
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
#lematize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
#term vector and tfidf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Import libraries for feature selection - Filter method
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#traintest split
from sklearn.model_selection import train_test_split

#Models
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,balanced_accuracy_score
from sklearn import metrics

pd.set_option('display.max_colwidth', -2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
#Read files
textfile = r'/gdrive/My Drive/CIS 508 Python/Assignment-6/Comments.csv'
textData = pd.read_csv(textfile) #creates a dataframe

CustInfofile = r'/gdrive/My Drive/CIS 508 Python/Assignment-6/Customers.csv'
CustInfoData = pd.read_csv(CustInfofile)  #creates a dataframe

print(textData.shape)
print(CustInfoData.shape)
textData

(2070, 2)
(2070, 17)


,ID,Comments
0,1309,Does not like the way the phone works. It is to difficult compared to his last phone.
1,3556,Wanted to know the nearest store location. Wants to buy aditional accessories.
2,2230,Wants to know how to do text messaging. Referred him to website.
3,2312,Asked how to disable call waiting. referred him to web site.
4,3327,Needs help learning how to use the phone. I suggested he go back to the store and have the rep teach him.
...,...,...
2065,3034,Needed help figuring out his bill. I explained our minute charges.
2066,271,He lost his phone and called to cancel service. I told him we would suspend until we hear back from him. He will contact us soon.
2067,783,Lost the directions to phone and wants another manual. I referred him to web site.
2068,1295,Wants to change address.


In [4]:
#Extract target column from Customer Info file
y_train = CustInfoData["TARGET"]
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
                     
print(X_train.shape)
print(textData.shape)
textData.head()
print(y_train)

(2070, 16)
(2070, 2)
0       Cancelled
1       Current  
2       Current  
3       Current  
4       Cancelled
          ...    
2065    Cancelled
2066    Cancelled
2067    Cancelled
2068    Cancelled
2069    Cancelled
Name: TARGET, Length: 2070, dtype: object


# Word Tokenize

In [5]:
#tokenise the sentences into words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)
print(textData['CommentsTokenized'])

0       [Does, not, like, the, way, the, phone, works, ., It, is, to, difficult, compared, to, his, last, phone, .]                                                       
1       [Wanted, to, know, the, nearest, store, location, ., Wants, to, buy, aditional, accessories, .]                                                                   
2       [Wants, to, know, how, to, do, text, messaging, ., Referred, him, to, website, .]                                                                                 
3       [Asked, how, to, disable, call, waiting, ., referred, him, to, web, site, .]                                                                                      
4       [Needs, help, learning, how, to, use, the, phone, ., I, suggested, he, go, back, to, the, store, and, have, the, rep, teach, him, .]                              
                                                                        ...                                                                      

# Lancaster Stemming

In [7]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -2)
#Lancaster stemming
# Use English stemmer.
lancaster=LancasterStemmer()
newTextData=pd.DataFrame()
newTextData=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData['CommentsTokenizedStemmedLancaster'] = textData['CommentsTokenized'].apply(lambda x: [lancaster.stem(y) for y in x])
print('-------------Lancaster Stemming------------------')
newTextData['CommentsTokenizedStemmedLancaster']
#Join stemmed strings
newTextData['CommentsTokenizedStemmedLancaster'] = newTextData['CommentsTokenizedStemmedLancaster'].apply(lambda x: " ".join(x))
newTextData['CommentsTokenizedStemmedLancaster']

-------------Lancaster Stemming------------------


0       doe not lik the way the phon work . it is to difficult comp to his last phon .                                              
1       want to know the nearest stor loc . want to buy adit access .                                                               
2       want to know how to do text mess . refer him to websit .                                                                    
3       ask how to dis cal wait . refer him to web sit .                                                                            
4       nee help learn how to us the phon . i suggest he go back to the stor and hav the rep teach him .                            
                                                      ...                                                                           
2065    nee help fig out his bil . i explain our minut charg .                                                                      
2066    he lost his phon and cal to cancel serv . i told him we would

# Creating Vector - Lancaster

In [8]:
#Do Bag-Of-Words model - Term - Document Matrix - Lancaster
#Learn the vocabulary dictionary and return term-document matrix.
#count_vect = CountVectorizer(stop_words=None)
count_vect = CountVectorizer(stop_words='english',lowercase=False)
count_vect
TD_counts = count_vect.fit_transform(newTextData.CommentsTokenizedStemmedLancaster)
print("Vocabulary ",(count_vect.vocabulary_))
print("Number of words in vocabulary",len(count_vect.vocabulary_))
print("Shape of text vector",TD_counts.shape)
DF_TD_Counts=pd.DataFrame(TD_counts.toarray(), columns= count_vect.get_feature_names())
print(DF_TD_Counts)

Vocabulary  {'doe': 96, 'lik': 173, 'way': 341, 'phon': 224, 'work': 356, 'difficult': 92, 'comp': 58, 'want': 339, 'know': 167, 'nearest': 201, 'stor': 283, 'loc': 176, 'buy': 40, 'adit': 9, 'access': 3, 'text': 303, 'mess': 189, 'refer': 243, 'websit': 344, 'ask': 23, 'dis': 95, 'cal': 41, 'wait': 338, 'web': 343, 'sit': 270, 'nee': 202, 'help': 144, 'learn': 171, 'suggest': 289, 'hav': 141, 'rep': 245, 'teach': 298, 'new': 205, 'plan': 226, 'switch': 296, 'soon': 276, 'mor': 196, 'minut': 191, 'addit': 6, 'ory': 213, 'said': 256, 'battery': 27, 'nev': 204, 'wel': 346, 'asap': 22, 'claim': 54, 'charg': 50, 'real': 238, 'result': 248, 'alway': 15, 'dying': 101, 'cur': 79, 'contract': 70, 'batery': 26, 'chang': 49, 'ring': 251, 'ton': 317, 'lost': 180, 'direct': 94, 'anoth': 18, 'man': 186, 'numb': 207, 'becaus': 28, 'mr': 198, 'napeleon': 199, 'leroy': 172, 'expect': 113, 'sign': 266, 'bet': 30, 'techn': 299, 'support': 291, 'pleas': 227, 'tak': 297, 'outbound': 216, 'list': 175, 'rat

# TF-IDF Matrix 

In [9]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(DF_TD_Counts)
print(X_train_tfidf.shape)
DF_TF_IDF=pd.DataFrame(X_train_tfidf.toarray(),columns= DF_TD_Counts.columns)
print(DF_TF_IDF)

(2070, 364)
      3399   3g  abysm    access   ad  ...  wrong  xvyx  year   yo  york
0     0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 
1     0.0   0.0  0.0    0.275669  0.0  ...  0.0    0.0   0.0   0.0  0.0 
2     0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 
3     0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 
4     0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 
...   ...   ...  ...         ...  ...  ...  ...    ...   ...   ...  ... 
2065  0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 
2066  0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 
2067  0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 
2068  0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 
2069  0.0   0.0  0.0    0.000000  0.0  ...  0.0    0.0   0.0   0.0  0.0 

[2070 rows x 364 columns]


# Feature Selection - Filter 

In [10]:
#Feature selection
selector = SelectKBest(score_func=chi2, k=30)
selector_fit = selector.fit(DF_TF_IDF,y_train)
#get scores along with features
names = DF_TF_IDF.columns.values[selector.get_support()]
scores = selector.scores_[selector.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns= ['Feat_names','F_Scores'])
ns_df_sorted = ns_df.sort_values(['F_Scores','Feat_names'], ascending = [False, True])
print(ns_df_sorted)


new_DF_TF_IDF = selector.fit_transform(DF_TF_IDF,y_train)
print("Shape of TF-IDF after feature selection", new_DF_TF_IDF.shape)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF, columns = list(DF_TF_IDF.columns[selector.get_support(indices=True)]) )
print(DF_TF_IDF_SelectedFeatures)

   Feat_names  F_Scores
26  transeff   3.570832
2   alway      3.345592
24  screening  2.955150
18  ot         2.567782
27  turn       2.567782
4   continu    2.439831
9   fig        2.193511
7   explain    1.995882
13  market     1.790824
19  peopl      1.790824
23  rid        1.790824
25  sold       1.790824
28  unlimit    1.707520
8   famy       1.631145
10  hochy      1.556822
15  momm       1.556822
14  minut      1.493673
5   cur        1.447126
6   dying      1.447126
22  result     1.447126
1   adress     1.446067
29  weak       1.362714
21  rel        1.353019
3   chang      1.289874
0   address    1.276869
20  plan       1.262413
11  internet   1.261777
12  man        1.203869
16  mor        1.157278
17  num        1.127669
Shape of TF-IDF after feature selection (2070, 30)
       address  adress  alway     chang  ...  transeff  turn  unlimit  weak
0     0.000000  0.0     0.0    0.000000  ...  0.0       0.0   0.0      0.0 
1     0.000000  0.0     0.0    0.000000  ...  0.0    

In [11]:
#split data into training and test
#split the entire training data in training and test data
X_Train, X_Test, Y_Train, Y_Test = train_test_split( DF_TF_IDF_SelectedFeatures, y_train, test_size=0.20, random_state=42)
print("Initial shape for entire data:",DF_TF_IDF_SelectedFeatures.shape)
print("Shape of new training data:", X_Train.shape)
print("Shape of new test split data:", X_Test.shape)

Initial shape for entire data: (2070, 30)
Shape of new training data: (1656, 30)
Shape of new test split data: (414, 30)


# Classification model on only comments - Random Forest - Filter

In [12]:
#Construct a Random Forest Classifier on text data
rfc=RandomForestClassifier()
rfc.fit(X_Train,Y_Train)

#Prediction on training data
pred_rf=pd.DataFrame(rfc.predict(X_Train),columns=["Prediction"])
print("----------------------------Random Forest: Training Data------------------------------------------------\n")
print("Accuracy Score on training data using Random Forest:",accuracy_score(Y_Train,pred_rf["Prediction"]))
print("Confusion Matrix on training data using Random Forest\n", confusion_matrix(Y_Train,pred_rf["Prediction"]))

# prediction on test data
pred_rf=pd.DataFrame(rfc.predict(X_Test),columns=["Prediction"])
print("----------------------------Random Forest: Test Data------------------------------------------------\n")
print("Accuracy Score on test data using Random Forest:",accuracy_score(Y_Test,pred_rf["Prediction"]))
print("Balanced Accuracy Score on test data using Random Forest:",balanced_accuracy_score(Y_Test,pred_rf["Prediction"]))
print("Confusion Matrix on test data using Random Forest\n", confusion_matrix(Y_Test,pred_rf["Prediction"]))
print("Classification report on test data using Random Forest\n", classification_report(Y_Test,pred_rf["Prediction"]))


----------------------------Random Forest: Training Data------------------------------------------------

Accuracy Score on training data using Random Forest: 0.6292270531400966
Confusion Matrix on training data using Random Forest
 [[ 60 587]
 [ 27 982]]
----------------------------Random Forest: Test Data------------------------------------------------

Accuracy Score on test data using Random Forest: 0.6280193236714976
Balanced Accuracy Score on test data using Random Forest: 0.520706832883095
Confusion Matrix on test data using Random Forest
 [[ 12 145]
 [  9 248]]
Classification report on test data using Random Forest
               precision    recall  f1-score   support

   Cancelled       0.57      0.08      0.13       157
     Current       0.63      0.96      0.76       257

    accuracy                           0.63       414
   macro avg       0.60      0.52      0.45       414
weighted avg       0.61      0.63      0.52       414



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


#customer data 

In [13]:
CustInfoData = CustInfoData.drop(columns=["TARGET"])
#Do one Hot encoding for categorical features
X_cat = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
#X_cat = CustInfoData.select_dtypes(include=['object'])
customer_one_hot = pd.get_dummies(CustInfoData,columns=X_cat)

customer_one_hot = pd.DataFrame(customer_one_hot)
print(customer_one_hot.head())

   ID  ...  LongDistanceBilltype_Standard
0  1   ...  0                            
1  6   ...  1                            
2  8   ...  1                            
3  11  ...  1                            
4  14  ...  0                            

[5 rows x 24 columns]


# Classification model on comments + customer data - Random Forest - Filter

In [16]:
#Merge files comments and customer data
combined=pd.concat([customer_one_hot, DF_TF_IDF_SelectedFeatures], axis=1)
print(combined.shape)
print(combined)

(2070, 54)
        ID  Children  Est_Income   Usage  ...  transeff  turn  unlimit  weak
0     1     1         38000.00    229.64  ...  0.0       0.0   0.0      0.0 
1     6     2         29616.00    75.29   ...  0.0       0.0   0.0      0.0 
2     8     0         19732.80    47.25   ...  0.0       0.0   0.0      0.0 
3     11    2         96.33       59.01   ...  0.0       0.0   0.0      0.0 
4     14    2         52004.80    28.14   ...  0.0       0.0   0.0      0.0 
...   ..   ..              ...      ...   ...  ...       ...   ...      ... 
2065  3821  0         78851.30    29.04   ...  0.0       0.0   0.0      0.0 
2066  3822  1         17540.70    36.20   ...  0.0       0.0   0.0      0.0 
2067  3823  0         83891.90    74.40   ...  0.0       0.0   0.0      0.0 
2068  3824  2         28220.80    38.95   ...  0.0       0.0   0.0      0.0 
2069  3825  0         28589.10    100.28  ...  0.0       0.0   0.0      0.0 

[2070 rows x 54 columns]


In [17]:
#split data into training and test
#split the entire training data in training and test data
X_Train, X_Test, Y_Train, Y_Test = train_test_split( combined, y_train, test_size=0.20, random_state=42)
print("Initial shape for entire data:",DF_TF_IDF_SelectedFeatures.shape)
print("Shape of new training data:", X_Train.shape)
print("Shape of new test split data:", X_Test.shape)

Initial shape for entire data: (2070, 30)
Shape of new training data: (1656, 54)
Shape of new test split data: (414, 54)


In [18]:
#Construct a Random Forest Classifier on combined data
rfc=RandomForestClassifier()
rfc.fit(X_Train,Y_Train)

#Prediction on training data
pred_rf=pd.DataFrame(rfc.predict(X_Train),columns=["Prediction"])
print("----------------------------Random Forest: Training Data------------------------------------------------\n")
print("Accuracy Score on training data using Random Forest:",accuracy_score(Y_Train,pred_rf["Prediction"]))
print("Confusion Matrix on training data using Random Forest\n", confusion_matrix(Y_Train,pred_rf["Prediction"]))

# prediction on test data
pred_rf=pd.DataFrame(rfc.predict(X_Test),columns=["Prediction"])
print("----------------------------Random Forest: Test Data------------------------------------------------\n")
print("Accuracy Score on test data using Random Forest:",accuracy_score(Y_Test,pred_rf["Prediction"]))
print("Balanced Accuracy Score on test data using Random Forest:",balanced_accuracy_score(Y_Test,pred_rf["Prediction"]))
print("Confusion Matrix on test data using Random Forest\n", confusion_matrix(Y_Test,pred_rf["Prediction"]))
print("Classification report on test data using Random Forest\n", classification_report(Y_Test,pred_rf["Prediction"]))

----------------------------Random Forest: Training Data------------------------------------------------

Accuracy Score on training data using Random Forest: 0.9909420289855072
Confusion Matrix on training data using Random Forest
 [[ 641    6]
 [   9 1000]]
----------------------------Random Forest: Test Data------------------------------------------------

Accuracy Score on test data using Random Forest: 0.8599033816425121
Balanced Accuracy Score on test data using Random Forest: 0.8499838905549084
Confusion Matrix on test data using Random Forest
 [[127  30]
 [ 28 229]]
Classification report on test data using Random Forest
               precision    recall  f1-score   support

   Cancelled       0.82      0.81      0.81       157
     Current       0.88      0.89      0.89       257

    accuracy                           0.86       414
   macro avg       0.85      0.85      0.85       414
weighted avg       0.86      0.86      0.86       414



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# Feature selection - Wrapper method 

In [32]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
#Construct a Random Forest Classifier on text data
rfc=RandomForestClassifier()

sfs1 = SFS(rfc, 
           k_features=20, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(DF_TF_IDF,y_train, custom_feature_names= DF_TF_IDF.columns)
print("Top 20 feature", sfs1.k_feature_names_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 364 out of 364 | elapsed:    7.7s finished

[2019-12-12 23:30:59] Features: 1/20 -- score: 0.6231884057971014[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 363 out of 363 | elapsed:    8.7s finished

[2019-12-12 23:31:08] Features: 2/20 -- score: 0.6285024154589371[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1

Top 20 feature ('3399', 'abysm', 'access', 'ad', 'adapt', 'addit', 'additon', 'address', 'adit', 'adress', 'advert', 'alway', 'anoth', 'ar', 'battery', 'hav', 'momm', 'phon', 'support', 'want')


[Parallel(n_jobs=1)]: Done 345 out of 345 | elapsed:   10.5s finished

[2019-12-12 23:34:07] Features: 20/20 -- score: 0.6352657004830918

In [33]:
#top 20 features after Forward
print("Top 20 feature", sfs1.k_feature_names_)

DF_TF_IDF_SelectedFeatures = DF_TF_IDF.loc[:, DF_TF_IDF.columns.isin(list(sfs1.k_feature_names_))]
print("Shape of TF-IDF after feature selection", DF_TF_IDF_SelectedFeatures.shape)
print(DF_TF_IDF_SelectedFeatures)

Top 20 feature ('3399', 'abysm', 'access', 'ad', 'adapt', 'addit', 'additon', 'address', 'adit', 'adress', 'advert', 'alway', 'anoth', 'ar', 'battery', 'hav', 'momm', 'phon', 'support', 'want')
Shape of TF-IDF after feature selection (2070, 20)
      3399  abysm    access   ad  ...  momm      phon  support      want
0     0.0   0.0    0.000000  0.0  ...  0.0   0.311374  0.0      0.000000
1     0.0   0.0    0.275669  0.0  ...  0.0   0.000000  0.0      0.312446
2     0.0   0.0    0.000000  0.0  ...  0.0   0.000000  0.0      0.195324
3     0.0   0.0    0.000000  0.0  ...  0.0   0.000000  0.0      0.000000
4     0.0   0.0    0.000000  0.0  ...  0.0   0.241722  0.0      0.000000
...   ...   ...         ...  ...  ...  ...        ...  ...           ...
2065  0.0   0.0    0.000000  0.0  ...  0.0   0.000000  0.0      0.000000
2066  0.0   0.0    0.000000  0.0  ...  0.0   0.167537  0.0      0.000000
2067  0.0   0.0    0.000000  0.0  ...  0.0   0.188306  0.0      0.153017
2068  0.0   0.0    0.0000

In [34]:
#split data into training and test
#split the entire training data in training and test data
X_Train, X_Test, Y_Train, Y_Test = train_test_split( DF_TF_IDF_SelectedFeatures, y_train, test_size=0.20, random_state=42)
print("Initial shape for entire data:",DF_TF_IDF_SelectedFeatures.shape)
print("Shape of new training data:", X_Train.shape)
print("Shape of new test split data:", X_Test.shape)

Initial shape for entire data: (2070, 20)
Shape of new training data: (1656, 20)
Shape of new test split data: (414, 20)


# Classification on comments - Wrapper method

In [35]:
#Construct a Random Forest Classifier on text data
rfc=RandomForestClassifier()
rfc.fit(X_Train,Y_Train)

#Prediction on training data
pred_rf=pd.DataFrame(rfc.predict(X_Train),columns=["Prediction"])
print("----------------------------Random Forest: Training Data------------------------------------------------\n")
print("Accuracy Score on training data using Random Forest:",accuracy_score(Y_Train,pred_rf["Prediction"]))
print("Confusion Matrix on training data using Random Forest\n", confusion_matrix(Y_Train,pred_rf["Prediction"]))

# prediction on test data
pred_rf=pd.DataFrame(rfc.predict(X_Test),columns=["Prediction"])
print("----------------------------Random Forest: Test Data------------------------------------------------\n")
print("Accuracy Score on test data using Random Forest:",accuracy_score(Y_Test,pred_rf["Prediction"]))
print("Balanced Accuracy Score on test data using Random Forest:",balanced_accuracy_score(Y_Test,pred_rf["Prediction"]))
print("Confusion Matrix on test data using Random Forest\n", confusion_matrix(Y_Test,pred_rf["Prediction"]))
print("Classification report on test data using Random Forest\n", classification_report(Y_Test,pred_rf["Prediction"]))

----------------------------Random Forest: Training Data------------------------------------------------

Accuracy Score on training data using Random Forest: 0.6358695652173914
Confusion Matrix on training data using Random Forest
 [[ 84 563]
 [ 40 969]]
----------------------------Random Forest: Test Data------------------------------------------------

Accuracy Score on test data using Random Forest: 0.6135265700483091
Balanced Accuracy Score on test data using Random Forest: 0.5127512453840244
Confusion Matrix on test data using Random Forest
 [[ 15 142]
 [ 18 239]]
Classification report on test data using Random Forest
               precision    recall  f1-score   support

   Cancelled       0.45      0.10      0.16       157
     Current       0.63      0.93      0.75       257

    accuracy                           0.61       414
   macro avg       0.54      0.51      0.45       414
weighted avg       0.56      0.61      0.52       414



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# Classification on customers + comments - Wrapper method

In [25]:
#Merge files comments and customer data
combined=pd.concat([customer_one_hot, DF_TF_IDF_SelectedFeatures], axis=1)
print(combined.shape)
print(combined)

(2070, 44)
        ID  Children  Est_Income   Usage  ...  expect  momm  sign      want
0     1     1         38000.00    229.64  ...  0.0     0.0   0.0   0.000000
1     6     2         29616.00    75.29   ...  0.0     0.0   0.0   0.312446
2     8     0         19732.80    47.25   ...  0.0     0.0   0.0   0.195324
3     11    2         96.33       59.01   ...  0.0     0.0   0.0   0.000000
4     14    2         52004.80    28.14   ...  0.0     0.0   0.0   0.000000
...   ..   ..              ...      ...   ...  ...     ...   ...        ...
2065  3821  0         78851.30    29.04   ...  0.0     0.0   0.0   0.000000
2066  3822  1         17540.70    36.20   ...  0.0     0.0   0.0   0.000000
2067  3823  0         83891.90    74.40   ...  0.0     0.0   0.0   0.153017
2068  3824  2         28220.80    38.95   ...  0.0     0.0   0.0   0.324251
2069  3825  0         28589.10    100.28  ...  0.0     0.0   0.0   0.000000

[2070 rows x 44 columns]


In [26]:
#split data into training and test
#split the entire training data in training and test data
X_Train, X_Test, Y_Train, Y_Test = train_test_split( combined, y_train, test_size=0.20, random_state=42)
print("Initial shape for entire data:",DF_TF_IDF_SelectedFeatures.shape)
print("Shape of new training data:", X_Train.shape)
print("Shape of new test split data:", X_Test.shape)

Initial shape for entire data: (2070, 20)
Shape of new training data: (1656, 44)
Shape of new test split data: (414, 44)


In [27]:
#Construct a Random Forest Classifier on combined data
rfc=RandomForestClassifier()
rfc.fit(X_Train,Y_Train)

#Prediction on training data
pred_rf=pd.DataFrame(rfc.predict(X_Train),columns=["Prediction"])
print("----------------------------Random Forest: Training Data------------------------------------------------\n")
print("Accuracy Score on training data using Random Forest:",accuracy_score(Y_Train,pred_rf["Prediction"]))
print("Confusion Matrix on training data using Random Forest\n", confusion_matrix(Y_Train,pred_rf["Prediction"]))

# prediction on test data
pred_rf=pd.DataFrame(rfc.predict(X_Test),columns=["Prediction"])
print("----------------------------Random Forest: Test Data------------------------------------------------\n")
print("Accuracy Score on test data using Random Forest:",accuracy_score(Y_Test,pred_rf["Prediction"]))
print("Balanced Accuracy Score on test data using Random Forest:",balanced_accuracy_score(Y_Test,pred_rf["Prediction"]))
print("Confusion Matrix on test data using Random Forest\n", confusion_matrix(Y_Test,pred_rf["Prediction"]))
print("Classification report on test data using Random Forest\n", classification_report(Y_Test,pred_rf["Prediction"]))


----------------------------Random Forest: Training Data------------------------------------------------

Accuracy Score on training data using Random Forest: 0.9915458937198067
Confusion Matrix on training data using Random Forest
 [[644   3]
 [ 11 998]]
----------------------------Random Forest: Test Data------------------------------------------------

Accuracy Score on test data using Random Forest: 0.8526570048309179
Balanced Accuracy Score on test data using Random Forest: 0.8478648789313242
Confusion Matrix on test data using Random Forest
 [[130  27]
 [ 34 223]]
Classification report on test data using Random Forest
               precision    recall  f1-score   support

   Cancelled       0.79      0.83      0.81       157
     Current       0.89      0.87      0.88       257

    accuracy                           0.85       414
   macro avg       0.84      0.85      0.84       414
weighted avg       0.85      0.85      0.85       414



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
